# 基于python的数据大屏实现

In [1]:
import streamlit as st
from streamlit_echarts import st_echarts
from pyecharts import options as opts
from pyecharts.charts import Bar
from streamlit_echarts import st_pyecharts
import pandas as pd
import numpy as np

In [2]:
a=pd.read_csv(r"D:\try\shoes.csv")
a.sales=a.sales.str.replace("人付款","").astype(int)
#求出各个商品的销售额并把它并入到原始数据框中去
z1=a.sales*a.price
z1.name="xse"
a=pd.concat([a,z1],axis=1)#给序列命名之后添加入数据框就会直接以序列名作为列标
location=a.location.str.split(" ",expand=True)
a["province"]=location[0]
a["city"]=location[1]

## 数据可视化展示的思路

* 首先每一行的意义

* 分析的目的性。对于商业分析而言，销售量，销售额始终是最优先需要展示的。各个部分的销售额，销售额的变化。与销售额密切相关的量，比如价格，销售量，点击行为，转化率....

* 特征与实体的关系。特征的分类，特征的意义在于解释销售额的来源，特征更像是x，销售额是y,商业上通常是通过调整x来实现y的最大化


* 以男鞋数据为例。每一行是搜索结果中的某个商品，因此所有size都是求商品数量，不同的groupby无非就是求不同特征相的商品数量。数据里给出了每个商品的销售量与价格，显然能获取到商品的销售额，因此一般来说我们所有的分析与可视化展现就应该仅仅围绕销售额和销售量展开。这个数据里，商品是一个天然的实体，有非常多的特征是从属于商品这个实体的；另外的实体就是店铺，但是我们可以看到数据里没有直接给出任何店铺的特征，但是店铺拥有商品，显然通过对商品的统计可以获取对应的店铺特征；此外商品的特征有几大类：首先是材质，风格，颜色等等商品的固有特征，此外则是省市这样的地域特征，然后是价格这个与销售额直接相关的特征(注意有时价格从属于订单而不是商品)。

* 基于以上考虑，我们很容易能够得到，如果希望从整体上展示这个数据整体情况应该从以下几个方面入手：销量的地域分布；；店铺销量排名；不同类型男鞋的销量；价格与销量

In [3]:
a.columns

Index(['_id.$oid', 'info.上市年份季节', 'info.上市时间', 'info.产品名称', 'info.低帮鞋品名',
       'info.功能', 'info.吊牌价', 'info.品牌', 'info.图案', 'info.场合', 'info.外底材料',
       'info.季节', 'info.尺码', 'info.帮面内里材质', 'info.帮面材质', 'info.性别',
       'info.是否商场同款', 'info.是否瑕疵', 'info.款号', 'info.款式', 'info.流行元素',
       'info.真皮材质工艺', 'info.细分风格', 'info.货号', 'info.跟底款式', 'info.运动系列',
       'info.运动鞋科技', 'info.适合路面', 'info.适用对象', 'info.销售渠道类型', 'info.闭合方式',
       'info.靴子品名', 'info.靴筒内里材质', 'info.靴筒材质', 'info.靴筒高度', 'info.鞋制作工艺',
       'info.鞋垫材质', 'info.鞋头款式', 'info.鞋底材质', 'info.鞋码', 'info.鞋跟高',
       'info.鞋面内里材质', 'info.鞋面材质', 'info.颜色分类', 'info.风格', 'itemid',
       'location', 'nick', 'price', 'sales', 'title', 'url', 'xse', 'province',
       'city'],
      dtype='object')

In [6]:
a.isnull().sum().head()

_id.$oid          0
info.上市年份季节    4483
info.上市时间      4986
info.产品名称      4978
info.低帮鞋品名     4512
dtype: int64

## 地域的数量分布

* 此数据里地域的销售额差异太大，已经不方便用地图展示了，因此这里只是展示商品数量
* 请注意各种可视化方案的容量和能忍受的数量差异的大小



* classwork 1

* 分布求出各个省与直辖市的商品数量；浙江省下属各个市的商品数量

* 在页面上可视化上述两个地域数量分布

* 实现下钻：点击全国地图中的省则显示对应省的各个市的商品数量

In [20]:
a.groupby("province").size()

province
上海     154
北京     339
吉林       1
四川       4
安徽      24
山东      44
山西      39
广东     284
江苏     120
江西      42
河北      86
河南     393
浙江    2470
湖北      27
福建     952
贵州       2
辽宁       3
重庆       3
dtype: int64

In [28]:
a[a["province"]=="浙江"].groupby("city").size().sort_values(ascending=False)

city
温州    1275
丽水     938
湖州     118
杭州     107
温岭      10
宁波       8
嘉兴       4
金华       4
衢州       3
台州       2
绍兴       1
dtype: int64

## 店铺销量排名与帕累托图

* 帕累托图，是“二八原则”的图形化体现。帕累托图分析可以帮助从多项因素中快速科学地找出最重要因素，便于研究者提出更有针对性的建议和解决措施



* classwork 2

* 获取各个店铺的销售额，并从大到小排序；获取店铺销售额的累计比率增长情况

* 在页面上实现帕累托图

In [11]:
nick_xse=a.groupby("nick").xse.sum().sort_values(ascending=False)
nick_xse=nick_xse[nick_xse>10000]

In [12]:
nick_xse_cum=nick_xse.cumsum()/nick_xse.sum()
nick_xse_cum

nick
意尔康皮鞋旗舰店           0.721249
意尔康男鞋旗舰店           0.881916
意尔康品牌店             0.905114
乙方乙方88888          0.922979
吸引力xl              0.931871
稚尚鞋类专营店            0.940513
夏克利2013            0.947934
因为有所以更好            0.953603
特别20150728         0.958676
新华鞋业168            0.963414
艾尚美梦工厂             0.967519
甜美果果t              0.970992
淡忘ing1027          0.974322
慢慢大王93             0.977412
zhanghaihuihiok    0.980400
chen356307796      0.983363
迪伯伦男鞋专家            0.986146
六六八鞋城              0.988746
tb877572_00        0.991205
温州女鞋店铺             0.993555
csdawww            0.995752
榴莲的淡香              0.997881
奥儿尼                1.000000
Name: xse, dtype: float64

In [32]:
nick_xse=a.groupby("nick").xse.sum().sort_values(ascending=False)

In [35]:
nick_xse

nick
意尔康皮鞋旗舰店     3558446.0
意尔康男鞋旗舰店      792685.0
意尔康品牌店        114452.0
乙方乙方88888      88144.0
吸引力xl          43870.0
               ...    
yek108             0.0
热血无极限              0.0
燕轩品牌店              0.0
爱鞋坊2012            0.0
龙凤吉祥4249           0.0
Name: xse, Length: 482, dtype: float64

In [37]:
nick_xse.cumsum()/nick_xse.sum()

nick
意尔康皮鞋旗舰店     0.647391
意尔康男鞋旗舰店     0.791604
意尔康品牌店       0.812427
乙方乙方88888    0.828463
吸引力xl        0.836444
               ...   
yek108       1.000000
热血无极限        1.000000
燕轩品牌店        1.000000
爱鞋坊2012      1.000000
龙凤吉祥4249     1.000000
Name: xse, Length: 482, dtype: float64

## 价格与销量

* 价格作为一个连续变量，通常用分段分组统计会更加合理



* classwork3

* 请求出各个区间的商品数量以及商品销量

* 请在页面上通过线图展现不同价格区间的数量与销量，注意这是双数值轴

In [13]:
a.price

0       269.0
1       199.0
2       148.0
3       158.0
4       279.0
        ...  
4982    119.0
4983    253.0
4984    187.0
4985    177.0
4986    213.0
Name: price, Length: 4987, dtype: float64

In [17]:
#np.linspace(a.price.min(),a.price.max(),500)

In [24]:
y1,x1=np.histogram(a.price.values,np.linspace(a.price.min(),a.price.max(),500),weights=a.sales.values)#商品销量

In [25]:
y2,x2=np.histogram(a.price.values,np.linspace(a.price.min(),a.price.max(),500))#商品数量

In [26]:
data1=[]
for i in range(0,len(y1)):
    data1.append([x1[i],y1[i]])

In [28]:
data2=[]
for i in range(0,len(y2)):
    data1.append([x2[i],y2[i]])

In [21]:
len(x2)

500

In [22]:
len(y2)

499

In [51]:
data1=[]
for i in range(0,len(y1)):
    data1.append([x1[i+1],y1[i]])

In [55]:
data1[0:2]

[[30.892184368737475, 686], [44.88436873747495, 0]]

In [ ]:
data1=[]
for i in range(0,len(y1)):
    data1.append([x1[i+1],y1[i]])

In [7]:
#data1

In [43]:
#x1

## 男鞋各个特征下的销量

* classwork 4

* "info.鞋面材质" "info.风格" "info.流行元素" 下的销售额

* 做饼图可视化到页面（饼图的容量很小，建议去掉尾部特征）

In [29]:
a.columns

Index(['_id.$oid', 'info.上市年份季节', 'info.上市时间', 'info.产品名称', 'info.低帮鞋品名',
       'info.功能', 'info.吊牌价', 'info.品牌', 'info.图案', 'info.场合', 'info.外底材料',
       'info.季节', 'info.尺码', 'info.帮面内里材质', 'info.帮面材质', 'info.性别',
       'info.是否商场同款', 'info.是否瑕疵', 'info.款号', 'info.款式', 'info.流行元素',
       'info.真皮材质工艺', 'info.细分风格', 'info.货号', 'info.跟底款式', 'info.运动系列',
       'info.运动鞋科技', 'info.适合路面', 'info.适用对象', 'info.销售渠道类型', 'info.闭合方式',
       'info.靴子品名', 'info.靴筒内里材质', 'info.靴筒材质', 'info.靴筒高度', 'info.鞋制作工艺',
       'info.鞋垫材质', 'info.鞋头款式', 'info.鞋底材质', 'info.鞋码', 'info.鞋跟高',
       'info.鞋面内里材质', 'info.鞋面材质', 'info.颜色分类', 'info.风格', 'itemid',
       'location', 'nick', 'price', 'sales', 'title', 'url', 'xse', 'province',
       'city'],
      dtype='object')

In [37]:
shoe_xm=a.groupby('info.鞋面材质').xse.sum().sort_values(ascending=False)
shoe_fg=a.groupby('info.风格').xse.sum().sort_values(ascending=False)
shoe_fg=a.groupby('info.流行元素').xse.sum().sort_values(ascending=False)

In [41]:
shoe_fg[0:5]

info.流行元素
车缝线     2877415.92
金属       973039.80
素面       375973.34
皮革拼接      68711.00
铆钉        47711.00
Name: xse, dtype: float64

In [40]:
data_fg=[]
for i in shoe_fg.items():
    data_fg.append({"value":i[1],"name":i[0]})

In [3]:
shoes_xiemian=a.groupby("info.鞋面材质").xse.sum().sort_values(ascending=False)

In [5]:
data_xiemian=[]
for i in shoes_xiemian.items():
    data_xiemian.append({"name":i[0],"value":i[1]})

In [9]:
data_xiemian[0:8]

[{'name': '头层牛皮（除牛反绒）', 'value': 5344427.22},
 {'name': '二层牛皮（除牛反绒）', 'value': 53818.4},
 {'name': '多种材质拼接', 'value': 37366.0},
 {'name': 'PU', 'value': 18503.5},
 {'name': '牛反绒', 'value': 9223.0},
 {'name': '网布', 'value': 6536.0},
 {'name': '二层猪皮', 'value': 3430.0},
 {'name': '头层猪皮', 'value': 2409.0}]

In [13]:
a.groupby("info.风格").xse.sum().sort_values(ascending=False)

info.风格
商务      3122703.94
休闲      2035391.58
青春潮流     192161.80
运动        37238.00
英伦        33702.00
韩版        24796.34
简约        19027.50
复古         1893.00
欧美         1017.00
民族风           0.00
Name: xse, dtype: float64

In [12]:
a.groupby("info.流行元素").xse.sum().sort_values(ascending=False)

info.流行元素
车缝线     2877415.92
金属       973039.80
素面       375973.34
皮革拼接      68711.00
铆钉        47711.00
雕花        43632.00
镂空        30223.00
亮片        13250.00
图腾        11633.40
绣花         4239.40
编制         3296.00
皮草装饰       2548.00
马衔扣        1730.00
奢华马毛       1026.00
迷彩            0.00
翻边            0.00
流苏            0.00
Name: xse, dtype: float64

In [4]:
a.columns

Index(['_id.$oid', 'info.上市年份季节', 'info.上市时间', 'info.产品名称', 'info.低帮鞋品名',
       'info.功能', 'info.吊牌价', 'info.品牌', 'info.图案', 'info.场合', 'info.外底材料',
       'info.季节', 'info.尺码', 'info.帮面内里材质', 'info.帮面材质', 'info.性别',
       'info.是否商场同款', 'info.是否瑕疵', 'info.款号', 'info.款式', 'info.流行元素',
       'info.真皮材质工艺', 'info.细分风格', 'info.货号', 'info.跟底款式', 'info.运动系列',
       'info.运动鞋科技', 'info.适合路面', 'info.适用对象', 'info.销售渠道类型', 'info.闭合方式',
       'info.靴子品名', 'info.靴筒内里材质', 'info.靴筒材质', 'info.靴筒高度', 'info.鞋制作工艺',
       'info.鞋垫材质', 'info.鞋头款式', 'info.鞋底材质', 'info.鞋码', 'info.鞋跟高',
       'info.鞋面内里材质', 'info.鞋面材质', 'info.颜色分类', 'info.风格', 'itemid',
       'location', 'nick', 'price', 'sales', 'title', 'url', 'xse', 'province',
       'city'],
      dtype='object')

In [7]:
p=a.isnull().sum()
p[p<1000]

_id.$oid         0
info.功能        403
info.品牌          5
info.图案        209
info.场合        312
info.季节        273
info.尺码          1
info.款式         14
info.流行元素      480
info.细分风格      809
info.货号        751
info.跟底款式      243
info.适用对象      781
info.闭合方式        6
info.鞋制作工艺     192
info.鞋头款式       12
info.鞋底材质      218
info.鞋跟高       799
info.鞋面内里材质     44
info.鞋面材质        7
info.颜色分类        0
info.风格        183
itemid           0
location         0
nick             0
price            0
sales            0
title            0
url              0
xse              0
province         0
city           496
dtype: int64

## 数据指标的展示

* st.columns
* st.metric

### classwork 5

请计算总商品数，总店铺数，总销售额，笔单价等量，然后并列展示在页面最上端

In [3]:
len(a)

4987

In [6]:
len(a.nick.unique())

482

In [9]:
round(a.xse.sum())

5496597

In [11]:
round(a.xse.sum()/a.sales.sum())

231

## 页面布局与图片设置

### classwork 6

1, 请通过st.set_page_config设置宽屏

2, 请通过st.columns调整各个图表的布局

3，通过st_echarts中的（height=625,width=800,theme='dark'）等参数调整图表的大小与主题

In [ ]:
st.set_page_config(page_title="column嵌套演示", layout="wide")

## 设置背景风格，字体颜色，和其他内容

* st.markdown

https://www.youtube.com/watch?v=pyWqw5yCNdo

https://discuss.streamlit.io/t/change-metric-color-font-background-and-style/25309


### classwork 7

1, 通过st.markdown加入大屏的标题

2，设置背景颜色

3，通过css设置好看的背景

https://www.magicpattern.design/

4，引入本地图片作为背景（G:\anaconda\Lib\site-packages\streamlit\static）

5,  通过定位页面元素，让最上面的导航栏透明；改变指标字体的颜色

In [ ]:
st.markdown("<h1 style='text-align: center; color: white;'>意尔康男鞋数据分析</h1>", unsafe_allow_html=True)

In [ ]:
#  background-image: url("data:image/svg+xml;utf8,%3Csvg viewBox=%220 0 2000 1000%22 xmlns=%22http:%2F%2Fwww.w3.org%2F2000%2Fsvg%22%3E%3Cmask id=%22b%22 x=%220%22 y=%220%22 width=%222000%22 height=%221000%22%3E%3Cpath fill=%22url(%23a)%22 d=%22M0 0h2000v1000H0z%22%2F%3E%3C%2Fmask%3E%3Cpath fill=%22%23000336%22 d=%22M0 0h2000v1000H0z%22%2F%3E%3Cg style=%22transform-origin:center center%22 stroke=%22%234c4e72%22 stroke-width=%222%22 mask=%22url(%23b)%22%3E%3Cpath fill=%22none%22 d=%22M0 0h100v100H0zM100 0h100v100H100zM200 0h100v100H200zM300 0h100v100H300z%22%2F%3E%3Cpath fill=%22%234c4e72a6%22 d=%22M400 0h100v100H400z%22%2F%3E%3Cpath fill=%22none%22 d=%22M500 0h100v100H500zM600 0h100v100H600z%22%2F%3E%3Cpath fill=%22%234c4e72f5%22 d=%22M700 0h100v100H700z%22%2F%3E%3Cpath fill=%22%234c4e72ef%22 d=%22M800 0h100v100H800z%22%2F%3E%3Cpath fill=%22%234c4e7296%22 d=%22M900 0h100v100H900z%22%2F%3E%3Cpath fill=%22none%22 d=%22M1000 0h100v100h-100zM1100 0h100v100h-100zM1200 0h100v100h-100z%22%2F%3E%3Cpath fill=%22%234c4e7297%22 d=%22M1300 0h100v100h-100z%22%2F%3E%3Cpath fill=%22none%22 d=%22M1400 0h100v100h-100zM1500 0h100v100h-100zM1600 0h100v100h-100z%22%2F%3E%3Cpath fill=%22%234c4e7290%22 d=%22M1700 0h100v100h-100z%22%2F%3E%3Cpath fill=%22none%22 d=%22M1800 0h100v100h-100zM1900 0h100v100h-100z%22%2F%3E%3Cpath fill=%22%234c4e72f6%22 d=%22M0 100h100v100H0z%22%2F%3E%3Cpath fill=%22%234c4e7274%22 d=%22M100 100h100v100H100z%22%2F%3E%3Cpath fill=%22%234c4e7232%22 d=%22M200 100h100v100H200z%22%2F%3E%3Cpath fill=%22none%22 d=%22M300 100h100v100H300zM400 100h100v100H400zM500 100h100v100H500zM600 100h100v100H600z%22%2F%3E%3Cpath fill=%22%234c4e7279%22 d=%22M700 100h100v100H700z%22%2F%3E%3Cpath fill=%22none%22 d=%22M800 100h100v100H800z%22%2F%3E%3Cpath fill=%22%234c4e72d5%22 d=%22M900 100h100v100H900z%22%2F%3E%3Cpath fill=%22%234c4e72a3%22 d=%22M1000 100h100v100h-100z%22%2F%3E%3Cpath fill=%22none%22 d=%22M1100 100h100v100h-100zM1200 100h100v100h-100zM1300 100h100v100h-100zM1400 100h100v100h-100zM1500 100h100v100h-100zM1600 100h100v100h-100z%22%2F%3E%3Cpath fill=%22%234c4e7212%22 d=%22M1700 100h100v100h-100z%22%2F%3E%3Cpath fill=%22%234c4e728b%22 d=%22M1800 100h100v100h-100z%22%2F%3E%3Cpath fill=%22none%22 d=%22M1900 100h100v100h-100z%22%2F%3E%3Cpath fill=%22%234c4e72a5%22 d=%22M0 200h100v100H0z%22%2F%3E%3Cpath fill=%22none%22 d=%22M100 200h100v100H100z%22%2F%3E%3Cpath fill=%22%234c4e720e%22 d=%22M200 200h100v100H200z%22%2F%3E%3Cpath fill=%22none%22 d=%22M300 200h100v100H300z%22%2F%3E%3Cpath fill=%22%234c4e72d9%22 d=%22M400 200h100v100H400z%22%2F%3E%3Cpath fill=%22none%22 d=%22M500 200h100v100H500zM600 200h100v100H600zM700 200h100v100H700z%22%2F%3E%3Cpath fill=%22%234c4e7247%22 d=%22M800 200h100v100H800z%22%2F%3E%3Cpath fill=%22%234c4e72ea%22 d=%22M900 200h100v100H900z%22%2F%3E%3Cpath fill=%22none%22 d=%22M1000 200h100v100h-100zM1100 200h100v100h-100zM1200 200h100v100h-100zM1300 200h100v100h-100z%22%2F%3E%3Cpath fill=%22%234c4e723a%22 d=%22M1400 200h100v100h-100z%22%2F%3E%3Cpath fill=%22%234c4e728c%22 d=%22M1500 200h100v100h-100z%22%2F%3E%3Cpath fill=%22none%22 d=%22M1600 200h100v100h-100zM1700 200h100v100h-100z%22%2F%3E%3Cpath fill=%22%234c4e7288%22 d=%22M1800 200h100v100h-100z%22%2F%3E%3Cpath fill=%22none%22 d=%22M1900 200h100v100h-100z%22%2F%3E%3Cpath fill=%22%234c4e720e%22 d=%22M0 300h100v100H0z%22%2F%3E%3Cpath fill=%22%234c4e72de%22 d=%22M100 300h100v100H100z%22%2F%3E%3Cpath fill=%22none%22 d=%22M200 300h100v100H200zM300 300h100v100H300zM400 300h100v100H400z%22%2F%3E%3Cpath fill=%22%234c4e7292%22 d=%22M500 300h100v100H500z%22%2F%3E%3Cpath fill=%22none%22 d=%22M600 300h100v100H600zM700 300h100v100H700zM800 300h100v100H800zM900 300h100v100H900zM1000 300h100v100h-100z%22%2F%3E%3Cpath fill=%22%234c4e72c2%22 d=%22M1100 300h100v100h-100z%22%2F%3E%3Cpath fill=%22none%22 d=%22M1200 300h100v100h-100zM1300 300h100v100h-100z%22%2F%3E%3Cpath fill=%22%234c4e7269%22 d=%22M1400 300h100v100h-100z%22%2F%3E%3Cpath fill=%22%234c4e7245%22 d=%22M1500 300h100v100h-100z%22%2F%3E%3Cpath fill=%22none%22 d=%22M1600 300h100v100h-100z%22%2F%3E%3Cpath fill=%22%234c4e72c2%22 d=%22M1700 300h100v100h-100z%22%2F%3E%3Cpath fill=%22none%22 d=%22M1800 300h100v100h-100zM1900 300h100v100h-100z%22%2F%3E%3Cpath fill=%22%234c4e72df%22 d=%22M0 400h100v100H0z%22%2F%3E%3Cpath fill=%22%234c4e72be%22 d=%22M100 400h100v100H100z%22%2F%3E%3Cpath fill=%22%234c4e7210%22 d=%22M200 400h100v100H200z%22%2F%3E%3Cpath fill=%22%234c4e7266%22 d=%22M300 400h100v100H300z%22%2F%3E%3Cpath fill=%22%234c4e720c%22 d=%22M400 400h100v100H400z%22%2F%3E%3Cpath fill=%22none%22 d=%22M500 400h100v100H500zM600 400h100v100H600zM700 400h100v100H700zM800 400h100v100H800z%22%2F%3E%3Cpath fill=%22%234c4e72de%22 d=%22M900 400h100v100H900z%22%2F%3E%3Cpath fill=%22%234c4e7296%22 d=%22M1000 400h100v100h-100z%22%2F%3E%3Cpath fill=%22none%22 d=%22M1100 400h100v100h-100zM1200 400h100v100h-100zM1300 400h100v100h-100zM1400 400h100v100h-100z%22%2F%3E%3Cpath fill=%22%234c4e7290%22 d=%22M1500 400h100v100h-100z%22%2F%3E%3Cpath fill=%22none%22 d=%22M1600 400h100v100h-100zM1700 400h100v100h-100z%22%2F%3E%3Cpath fill=%22%234c4e7278%22 d=%22M1800 400h100v100h-100z%22%2F%3E%3Cpath fill=%22none%22 d=%22M1900 400h100v100h-100zM0 500h100v100H0zM100 500h100v100H100zM200 500h100v100H200z%22%2F%3E%3Cpath fill=%22%234c4e72a9%22 d=%22M300 500h100v100H300z%22%2F%3E%3Cpath fill=%22none%22 d=%22M400 500h100v100H400zM500 500h100v100H500zM600 500h100v100H600z%22%2F%3E%3Cpath fill=%22%234c4e7289%22 d=%22M700 500h100v100H700z%22%2F%3E%3Cpath fill=%22%234c4e72b6%22 d=%22M800 500h100v100H800z%22%2F%3E%3Cpath fill=%22none%22 d=%22M900 500h100v100H900zM1000 500h100v100h-100z%22%2F%3E%3Cpath fill=%22%234c4e72a5%22 d=%22M1100 500h100v100h-100z%22%2F%3E%3Cpath fill=%22none%22 d=%22M1200 500h100v100h-100z%22%2F%3E%3Cpath fill=%22%234c4e7237%22 d=%22M1300 500h100v100h-100z%22%2F%3E%3Cpath fill=%22none%22 d=%22M1400 500h100v100h-100zM1500 500h100v100h-100zM1600 500h100v100h-100zM1700 500h100v100h-100z%22%2F%3E%3Cpath fill=%22%234c4e7271%22 d=%22M1800 500h100v100h-100z%22%2F%3E%3Cpath fill=%22none%22 d=%22M1900 500h100v100h-100zM0 600h100v100H0zM100 600h100v100H100zM200 600h100v100H200z%22%2F%3E%3Cpath fill=%22%234c4e7270%22 d=%22M300 600h100v100H300z%22%2F%3E%3Cpath fill=%22none%22 d=%22M400 600h100v100H400zM500 600h100v100H500zM600 600h100v100H600zM700 600h100v100H700z%22%2F%3E%3Cpath fill=%22%234c4e725f%22 d=%22M800 600h100v100H800z%22%2F%3E%3Cpath fill=%22none%22 d=%22M900 600h100v100H900z%22%2F%3E%3Cpath fill=%22%234c4e722a%22 d=%22M1000 600h100v100h-100z%22%2F%3E%3Cpath fill=%22%234c4e729a%22 d=%22M1100 600h100v100h-100z%22%2F%3E%3Cpath fill=%22none%22 d=%22M1200 600h100v100h-100zM1300 600h100v100h-100z%22%2F%3E%3Cpath fill=%22%234c4e720f%22 d=%22M1400 600h100v100h-100z%22%2F%3E%3Cpath fill=%22none%22 d=%22M1500 600h100v100h-100zM1600 600h100v100h-100zM1700 600h100v100h-100z%22%2F%3E%3Cpath fill=%22%234c4e7241%22 d=%22M1800 600h100v100h-100z%22%2F%3E%3Cpath fill=%22%234c4e723e%22 d=%22M1900 600h100v100h-100z%22%2F%3E%3Cpath fill=%22%234c4e7276%22 d=%22M0 700h100v100H0z%22%2F%3E%3Cpath fill=%22none%22 d=%22M100 700h100v100H100z%22%2F%3E%3Cpath fill=%22%234c4e7297%22 d=%22M200 700h100v100H200z%22%2F%3E%3Cpath fill=%22none%22 d=%22M300 700h100v100H300zM400 700h100v100H400zM500 700h100v100H500zM600 700h100v100H600z%22%2F%3E%3Cpath fill=%22%234c4e729f%22 d=%22M700 700h100v100H700z%22%2F%3E%3Cpath fill=%22%234c4e7203%22 d=%22M800 700h100v100H800z%22%2F%3E%3Cpath fill=%22none%22 d=%22M900 700h100v100H900zM1000 700h100v100h-100z%22%2F%3E%3Cpath fill=%22%234c4e728a%22 d=%22M1100 700h100v100h-100z%22%2F%3E%3Cpath fill=%22none%22 d=%22M1200 700h100v100h-100zM1300 700h100v100h-100z%22%2F%3E%3Cpath fill=%22%234c4e7271%22 d=%22M1400 700h100v100h-100z%22%2F%3E%3Cpath fill=%22none%22 d=%22M1500 700h100v100h-100zM1600 700h100v100h-100zM1700 700h100v100h-100zM1800 700h100v100h-100z%22%2F%3E%3Cpath fill=%22%234c4e724e%22 d=%22M1900 700h100v100h-100z%22%2F%3E%3Cpath fill=%22none%22 d=%22M0 800h100v100H0z%22%2F%3E%3Cpath fill=%22%234c4e7294%22 d=%22M100 800h100v100H100z%22%2F%3E%3Cpath fill=%22%234c4e72ad%22 d=%22M200 800h100v100H200z%22%2F%3E%3Cpath fill=%22%234c4e728b%22 d=%22M300 800h100v100H300z%22%2F%3E%3Cpath fill=%22%234c4e7272%22 d=%22M400 800h100v100H400z%22%2F%3E%3Cpath fill=%22none%22 d=%22M500 800h100v100H500zM600 800h100v100H600zM700 800h100v100H700zM800 800h100v100H800zM900 800h100v100H900zM1000 800h100v100h-100zM1100 800h100v100h-100z%22%2F%3E%3Cpath fill=%22%234c4e72cd%22 d=%22M1200 800h100v100h-100z%22%2F%3E%3Cpath fill=%22none%22 d=%22M1300 800h100v100h-100z%22%2F%3E%3Cpath fill=%22%234c4e7256%22 d=%22M1400 800h100v100h-100z%22%2F%3E%3Cpath fill=%22%234c4e729a%22 d=%22M1500 800h100v100h-100z%22%2F%3E%3Cpath fill=%22none%22 d=%22M1600 800h100v100h-100zM1700 800h100v100h-100zM1800 800h100v100h-100z%22%2F%3E%3Cpath fill=%22%234c4e720b%22 d=%22M1900 800h100v100h-100z%22%2F%3E%3Cpath fill=%22none%22 d=%22M0 900h100v100H0zM100 900h100v100H100zM200 900h100v100H200zM300 900h100v100H300z%22%2F%3E%3Cpath fill=%22%234c4e72af%22 d=%22M400 900h100v100H400z%22%2F%3E%3Cpath fill=%22none%22 d=%22M500 900h100v100H500z%22%2F%3E%3Cpath fill=%22%234c4e7286%22 d=%22M600 900h100v100H600z%22%2F%3E%3Cpath fill=%22none%22 d=%22M700 900h100v100H700zM800 900h100v100H800zM900 900h100v100H900zM1000 900h100v100h-100zM1100 900h100v100h-100zM1200 900h100v100h-100z%22%2F%3E%3Cpath fill=%22%234c4e7231%22 d=%22M1300 900h100v100h-100z%22%2F%3E%3Cpath fill=%22none%22 d=%22M1400 900h100v100h-100z%22%2F%3E%3Cpath fill=%22%234c4e7208%22 d=%22M1500 900h100v100h-100z%22%2F%3E%3Cpath fill=%22none%22 d=%22M1600 900h100v100h-100z%22%2F%3E%3Cpath fill=%22%234c4e72ce%22 d=%22M1700 900h100v100h-100z%22%2F%3E%3Cpath fill=%22none%22 d=%22M1800 900h100v100h-100zM1900 900h100v100h-100z%22%2F%3E%3C%2Fg%3E%3Cdefs%3E%3CradialGradient id=%22a%22%3E%3Cstop offset=%220%22 stop-color=%22%23fff%22%2F%3E%3Cstop offset=%221%22 stop-color=%22%23fff%22 stop-opacity=%220%22%2F%3E%3C%2FradialGradient%3E%3C%2Fdefs%3E%3C%2Fsvg%3E");

In [ ]:
bg_img="""
<style>
[data-testid="stAppViewContainer"]{
  width: 100%;
  height: 100%;
  background-size: cover;
  background-position: center center;
  background-repeat: repeat;
  background-image: url("./xing.png")
    }

[data-testid="stHeader"]{
    background-color:rgba(0, 0, 0, 0)
    }

[data-testid="metric-container"] {
   background-color: rgba(28, 131, 225, 0.1);
   border: 1px solid rgba(28, 131, 225, 0.1);
   padding: 5% 5% 5% 10%;
   border-radius: 5px;
   color: rgb(30, 103, 119);
   overflow-wrap: break-word;
}

/* breakline for metric text         */
[data-testid="metric-container"] {
   overflow-wrap: break-word;
   white-space: break-spaces;
   color: white;
}
</style>
"""
st.markdown(bg_img, unsafe_allow_html=True)